In [95]:
from datetime import datetime ,timedelta                 
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)
import json
import re     
import numpy as np
import pickle

import matplotlib.pyplot as plt
import sys
import os
from pandas.io.json import json_normalize
import seaborn as sns
import statsmodels.api as sm
import math

In [3]:
train_24 = pd.read_csv('train_24.csv')
train_48 = pd.read_csv('train_48.csv')

In [8]:
train_24.shape

(346005, 10)

In [43]:
jy_nulldf_24 = pd.read_csv('data/tr_24_jy_null.csv')
jy_nulldf_48 = pd.read_csv('data/tr_48_jy_null.csv')

# ***Feature Engineering***

    - 기준온도 비교 %
    - 24시간 기준날짜, 48시간 기준날짜 10분전,20분,30분...1시간...2시간 전의 설명변수 변수화
    - 온도 interval + 습도 interval 교차 변수
    - 온도 특정 이상치에서 다른 변수들의 패턴 확인
    - 날씨변수들을 묶어서 클러스터링 군집화
    - 

In [9]:
# 기준온도 비교
base_tem = (train_24['plant_train.tem_in'] - train_24['plant_train.tem_coil']) / train_24['plant_train.tem_in']

In [17]:
date0 = str(datetime.strptime(train_24[['date','plant','loc']].iloc[0].values[0],'%Y-%m-%d %H:%M:%S'))

In [24]:
datetime.strptime(train_24[['date','plant','loc']].iloc[0].values[0],'%Y-%m-%d %H:%M:%S') + timedelta(hours = 23., )

datetime.datetime(2016, 4, 1, 23, 36)

In [10]:
# 기준날짜 이전의 time데이터 변수
def pre_date_variable(df,minute):
    
    pre_date = []
    date_ls = df['date'].tolist()
    for i in range(len(df)):
        
        df2 = df[['date','plant','loc']].iloc[i]
        
        date_after24 = datetime.strptime(df.values[0],'%Y-%m-%d %H:%M:%S') + timedelta(days = 1)
        
        date_after_str = str(date_after24 - timedelta(minutes = 10))
        
        try:
            idx = date_ls.index(date_after_str)
        
        except:
            print(date_ls[i] + '날짜는 존재x')
            continue
        
        
        
        
        

,after24_loc_predict,date,plant_train.cond_loc,plant_train.hum_in,plant_train.tem_coil,plant_train.tem_in,plant_train.tem_out_loc1,plant_train.hum_out_loc1,loc,plant
0,0.0,2016-04-01 00:00:00,0.0,24.00,11.00,16.00,9.00,42.00,1,1
1,0.0,2016-04-01 03:00:00,0.0,28.00,10.00,14.00,7.00,59.00,1,1
2,0.0,2016-04-01 06:00:00,0.0,33.00,10.00,13.00,6.00,56.00,1,1
3,0.0,2016-04-01 09:00:00,0.0,33.00,10.00,13.00,18.00,30.00,1,1
4,0.0,2016-04-01 12:00:00,0.0,28.00,10.00,16.00,18.00,20.00,1,1
...,...,...,...,...,...,...,...,...,...,...
346000,0.0,2019-03-30 23:00:00,0.0,54.18,10.25,8.72,5.05,73.32,3,2
346001,0.0,2019-03-30 23:10:00,0.0,54.67,10.29,8.78,5.12,74.05,3,2
346002,0.0,2019-03-30 23:20:00,0.0,54.67,10.29,8.75,5.18,72.65,3,2
346003,0.0,2019-03-30 23:30:00,0.0,55.01,10.21,8.75,5.24,70.91,3,2


In [ ]:
#온도 interval, 습도 interval 교호 변수
#온도 5 ~ 20도 이면서 습도 70이상 이면 1

# ***결측치 처리***

    - 결측치를 처리했을 때 데이터 / 결측치를 처리하지 않고 그대로 사용한 데이터의 성능 비교필요

***범주형 변수에 대한 처리도 필요***
    - dong_raintype, dong_thunder

In [32]:
# 시계열성 특징을 가지고 있고, 이상치가 크게 없으므로 결측치를 linear하게 처리해주는 함수
def nan_preprocessing(df,col):
    col_value = []
    for i in range(len(df)):
        if i != 0 and str(df[col].iloc[i]) == 'nan':

            if str(df[col].iloc[i-1]) == 'nan':
                pass

            else:

                for j in range(i+1,len(df)):
                    if str(df[col].iloc[j]) != 'nan':
                        interval = abs(df[col].iloc[i-1] - df[col].iloc[j])
                        cnt = j - (i-1)

                        value = interval / cnt 

                        value2 = df[col].iloc[i-1]

                        if df[col].iloc[i-1] < df[col].iloc[j]:
                            right_left = 'left'

                        else:
                            right_left = 'right'

                        for k in range(0,cnt - 1):

                            if right_left == 'left':
                                value2 += value
                                col_value.append(value2)
                            elif right_left == 'right':
                                value2 -= value
                                col_value.append(value2)

                        break


                    elif str(df[col].iloc[j]) == 'nan':
                        if j != len(df) - 1:
                            pass
                        elif j == len(df) - 1:
                            nan_len = j - i + 1
                            nan_value_ls = ['nan'] * nan_len

                            col_value = col_value + nan_value_ls
                
        else:
            col_value.append(df[col].iloc[i])
            
    return col_value

In [68]:
nan_col24 = []
for i in range(len(jy_nulldf_24.isnull().sum().values)):
    
    if jy_nulldf_24.isnull().sum().values[i] != 0:
        nan_col24.append(jy_nulldf_24.isnull().sum().keys()[i])
        
nan_col48 = []
for i in range(len(jy_nulldf_48.isnull().sum().values)):
    
    if jy_nulldf_48.isnull().sum().values[i] != 0:
        nan_col48.append(jy_nulldf_48.isnull().sum().keys()[i])

In [69]:
nan_col24 = set(nan_col24) - set(['dong_raintype', 'dong_thunder'])
nan_col48 = set(nan_col48) - set(['dong_raintype', 'dong_thunder'])

In [70]:
for i in nan_col24:
    jy_nulldf_24[i] = nan_preprocessing(jy_nulldf_24,i)

In [81]:
for i in nan_col48:
    jy_nulldf_48[i] = nan_preprocessing(jy_nulldf_48,i)
    print(i)

asos_tem
rain
wind_speed
cloud
sea_pres
sun
pres
dong_rain
sum_en
dong_tem
wind_direct
dong_winddrt
dong_windsp
asos_hum
dong_hum


In [82]:
jy_nulldf_48.columns

Index(['date', 'loc', 'plant', 'asos_tem', 'rain', 'wind_direct', 'wind_speed',
       'pres', 'sea_pres', 'asos_hum', 'sum_en', 'sun', 'month', 'season',
       'time', 'tem_26', 'tem_coil_510', 'tem_coil_25', 'hum_out_4361',
       'asos_hum_95', 'hum_tem_coil', 'plant_train.hum_in_log',
       'plant_train.tem_coil_log', 'plant_train.tem_in_log',
       'plant_train.tem_out_loc1_log', 'plant_train.hum_out_loc1_log',
       'asos_tem_log', 'rain_log', 'wind_direct_log', 'wind_speed_log',
       'pres_log', 'sea_pres_log', 'asos_hum_log', 'sum_en_log', 'sun_log',
       'dong_rain', 'dong_raintype', 'dong_tem', 'dong_thunder', 'dong_hum',
       'dong_windsp', 'dong_winddrt', 'cloud'],
      dtype='object')

# ***추가 데이터 목록***

- 태양고도, 방위각
- 그날 하루의 일출, 일몰
- 이슬점


In [ ]:
# 이슬점 구하는 공식
def dew_point(df,tem_column,hum_column):
    b = 17.62
    c = 243.12
    
    dew_point_ls = []
    
    for i in range(len(df)):

        gamma = (b * x_dj_nulldf_48[tem_column].iloc[i] /(c + x_dj_nulldf_48[tem_column].iloc[i])) + math.log(x_dj_nulldf_48[hum_column].iloc[i] / 100.0)
        dewpoint = (c * gamma) / (b - gamma)
        
        dew_point_ls.append(dewpoint)
        
    return dew_point_ls